In [21]:
text = readlines("inputb")
ind_lights = []
buttons = []
joltage = []
for (i,line) in enumerate(text)
    split_line = split(line, " ")
    new_buttons = []
    lightsize = length(split_line[1]) -2
    for (j, word) in enumerate(split_line)
        if j==1
            arr= zeros(Int,lightsize)
            for (k,char) in enumerate(word[2:end-1])
                if char == '#'
                    arr[k]=1
                end
            end
            push!(ind_lights, arr)

        elseif j == length(split_line)
            push!(joltage, word)
        else
            nums = parse.(Int, split(word[2:end-1], ","))
            new_nums = zeros(Int, lightsize)

            for num in nums
                new_nums[num+1] = 1
            end
            push!(new_buttons, new_nums)
        end
    end
    push!(buttons, new_buttons)
end
println(ind_lights)
println(buttons)
println(joltage)

Any[[0, 1, 1, 0, 1, 0, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 0, 1, 1], [1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 0, 0, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 1, 1], [1, 1, 0, 1], [1, 1, 0, 0, 1, 0], [0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 1, 0, 1, 1, 0], [0, 0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1, 1], [1, 0, 1, 1, 0, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0], [0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 1], [1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 1, 1, 1], [1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 1, 0, 1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 0, 1, 1],

In [22]:
using JuMP, HiGHS
function optimize_button_pressing(lights, buttons)
    d = length(lights)
    N = length(buttons)
    
    #println(lights)
    #println(buttons)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, k[1:N]>=0, Int)
    @objective(model, Min, sum(k))
    @variable(model, m[1:d], Int)
    for j in 1:d
        @constraint(model, sum(k[i]*buttons[i][j] for i in 1:N) -2*m[j]== lights[j])
    end

    optimize!(model)
    return sum(value.(k))
end



optimize_button_pressing (generic function with 1 method)

In [23]:
summ = 0
for i in 1:length(ind_lights)
    lights = ind_lights[i]
    btns = buttons[i]
    result = optimize_button_pressing(lights, btns)
    #println("Minimum button presses for light set $i: $result")
    summ+=result
end
println(summ)

449.0


In [ ]:
using JuMP, HiGHS
vt = ind_lights[1]
d = length(vt)
vs = buttons[1]
N = length(vs)

model = Model(HiGHS.Optimizer)
@variable(model, k[1:N]>=0, Int)
@objective(model, Min, sum(k))

for j in 1:d
    @constraint(model, sum(k[i]*vs[i][j] for i in 1:N) == vt[j])
end
optimize!(model)
println(value.(k))

Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 4 rows; 12 cols; 20 nonzeros; 12 integer variables (0 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+00, 1e+00]
  Bound   [0e+00, 0e+00]
  RHS     [1e+00, 1e+00]
Presolving model
4 rows, 12 cols, 20 nonzeros  0s
4 rows, 12 cols, 20 nonzeros  0s
Presolve reductions: rows 4(-0); columns 12(-0); nonzeros 20(-0) - Not reduced
Objective function is integral with scale 1

Solving MIP model with:
   4 rows
   12 cols (0 binary, 12 integer, 0 implied int., 0 continuous, 0 domain fixed)
   20 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

  